In [1]:
import pandas as pd
import os
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
folder_path = r'.\ml-1m' 
unames = ['user_id', 'gender', 'age', 'occupation', 'zip'] 
users = pd.read_table(os.path.join(folder_path,'users.dat'), sep='::', header=None, names=unames)
rnames = ['user_id', 'movie_id', 'rating',	'timestamp'] 
ratings = pd.read_table(os.path.join(folder_path,'ratings.dat'), sep = '::', header=None, names=rnames) 
mnames = ['movie_id', 'title', 'genres'] 
movies = pd.read_table(os.path.join(folder_path,'movies.dat'),sep='::',header=None,	names=mnames,engine='python', encoding='ISO-8859-1')

C:\Users\AlanJobs\AppData\Local\Temp\ipykernel_21828\1564436531.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  users = pd.read_table(os.path.join(folder_path,'users.dat'), sep='::', header=None, names=unames)
C:\Users\AlanJobs\AppData\Local\Temp\ipykernel_21828\1564436531.py:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  ratings = pd.read_table(os.path.join(folder_path,'ratings.dat'), sep = '::', header=None, names=rnames)


In [3]:
print(users[:5])
print(ratings[:5])
print(movies[:5])

   user_id gender  age  occupation    zip
0        1      F    1          10  48067
1        2      M   56          16  70072
2        3      M   25          15  55117
3        4      M   45           7  02460
4        5      M   25          20  55455
   user_id  movie_id  rating  timestamp
0        1      1193       5  978300760
1        1       661       3  978302109
2        1       914       3  978301968
3        1      3408       4  978300275
4        1      2355       5  978824291
   movie_id                               title                        genres
0         1                    Toy Story (1995)   Animation|Children's|Comedy
1         2                      Jumanji (1995)  Adventure|Children's|Fantasy
2         3             Grumpier Old Men (1995)                Comedy|Romance
3         4            Waiting to Exhale (1995)                  Comedy|Drama
4         5  Father of the Bride Part II (1995)                        Comedy


In [4]:
vectorizer = CountVectorizer(tokenizer=lambda x: x.split('|'))
genres_matrix = vectorizer.fit_transform(movies['genres'])
feature_names = vectorizer.get_feature_names_out()
print(feature_names)

['action' 'adventure' 'animation' "children's" 'comedy' 'crime'
 'documentary' 'drama' 'fantasy' 'film-noir' 'horror' 'musical' 'mystery'
 'romance' 'sci-fi' 'thriller' 'war' 'western']


c:\ProgramData\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [5]:
# 这是一个稀疏矩阵，含义是第 0 部电影中第 2 种类型出现了 1 次 ---> (0, 2) 1
print(genres_matrix.shape)
print(genres_matrix[:5])

(3883, 18)
  (0, 2)	1
  (0, 3)	1
  (0, 4)	1
  (1, 3)	1
  (1, 1)	1
  (1, 8)	1
  (2, 4)	1
  (2, 13)	1
  (3, 4)	1
  (3, 7)	1
  (4, 4)	1


In [6]:
#余弦相似度 [-1,1] (i,j)
cosine_sim = cosine_similarity(genres_matrix)
print(cosine_sim.shape)
print(cosine_sim)

(3883, 3883)
[[1.         0.33333333 0.40824829 ... 0.         0.         0.        ]
 [0.33333333 1.         0.         ... 0.         0.         0.        ]
 [0.40824829 0.         1.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 1.         1.         0.70710678]
 [0.         0.         0.         ... 1.         1.         0.70710678]
 [0.         0.         0.         ... 0.70710678 0.70710678 1.        ]]


In [8]:
def content_based_recommendation(user_id,number):

    #user_id评论过的电影
    user_movies = ratings[ratings['user_id'] == user_id]
    movie_indices = user_movies['movie_id'].values
    # 求出每个电影与评论过的电影的相似度的和（这里ppt可以详细画张图解释，比如我评论过了第i，j行，那么矩阵第一列（即第一个电影）的i,j行的和就是第一个电影和我评论过的电影的相似度和
    scores = cosine_sim[movie_indices].sum(axis=0)
    recommended_movie_indices = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)
    # 将矩阵的维度说明下
    #print(user_movies)
    #print((movie_indices)) 1用户评论过53个电影
    #print(len(scores))3883
    #print(recommended_movie_indices)
    recommended_movie_names=[]
    for index,i in enumerate(recommended_movie_indices):
        if(index==number):
            break
        recommended_movie_names.append(movies.loc[movies['movie_id'] == i, 'title'].values[0])
    return recommended_movie_names
user_id = 1
number = 5
print(content_based_recommendation(user_id,number))

['Grumpier Old Men (1995)', 'Mortal Kombat (1995)', 'Fair Game (1995)', 'Bed of Roses (1996)', 'Once Upon a Time... When We Were Colored (1995)']
